In [1]:
import random
import pickle
from collections import namedtuple 
from sklearn.model_selection import train_test_split
import numpy as np

Transition = namedtuple('Transition',['entity', 'property', 'value'])

In [2]:
with open('data/tuples_v2_with_add_info.pkl', 'rb') as f:
    d = pickle.load(f)
    data = d['data']
    topics_map = d['topics_map']
    all_transitions = d['transitions']

X, y = [], []

for k, v in data.items():
    y.append(topics_map[k])
    X.append(v)


In [3]:
# Get train, val, test split in ratio 0.75:0.1:0.15 of data_v
X_train, X_test, y_train, _ = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y, shuffle=True)
X_train, X_val = train_test_split(X_train, test_size=0.1, random_state=42, stratify=y_train, shuffle=True)


In [12]:
len(X_train), len(X_val), len(X_test)

(950, 106, 187)

In [13]:
def traverse(procedure):
    return [transition for step in procedure for transition in step]

def get_traversals(procedures):
    return [traverse(procedure) for procedure in procedures]
    

In [14]:
traverse([
    ['1.1'],
    ['2.1', '2.2'],
    ['3.1', '3.2', '3.3'],
    ['4.1']
])

['1.1', '2.1', '2.2', '3.1', '3.2', '3.3', '4.1']

In [15]:
get_traversals([
    [
        ['1.1'],
        ['2.1', '2.2'],
        ['3.1', '3.2', '3.3'],
    ],
    [
        ['1.1'],
        ['2.1', '2.2','2.3'],
    ]
])

[['1.1', '2.1', '2.2', '3.1', '3.2', '3.3'], ['1.1', '2.1', '2.2', '2.3']]

In [16]:
train = get_traversals(X_train)
val = get_traversals(X_val)
test = get_traversals(X_test)

In [17]:
# store the data
with open('processed_data/traversed_data.pkl', 'wb') as f:
    pickle.dump({'train': train, 'val': val, 'test': test}, f)

In [18]:
def _add_procedure_to_dataset(procedure, length, negatives, result):
    # print(procedure, length, negatives)
    assert 0 < length < len(procedure), "Length of procedure must be between one and one less than the length of the procedure"
    CANDIDATE_LEN = len(negatives) + 1
    negatives_length = max(0, len(negatives) - (len(procedure) - length) + 1)
    result.append({
        'given': procedure[:length],
        'next': (procedure[length:] + negatives[:negatives_length])[:CANDIDATE_LEN],
        'answer': 0,
    })


In [ ]:


def sampling(data, lower_bound=3, upper_bound=7, sampling_prob_1=0.25):
    # @NOTE: [1,1], [2,lower_bound], [lower_bound+1,upper_bound], [upper_bound,*]
    assert lower_bound >= 2
    random.seed(42)
    result = []
    for procedure in data:
        # randomly sample 5 transitions that is not in current procedure
        negative_transitions = [t for t in all_transitions if t not in set(procedure)]
        sampled_transitions = random.sample(negative_transitions, 5)
        proc_length = len(procedure)
        if proc_length <= 1:
            continue
        elif proc_length <= lower_bound:
            _add_procedure_to_dataset(procedure, proc_length - 1, sampled_transitions, result)
        else:
            for given_length in range(lower_bound - 1, proc_length - 1):
                if random.random() < sampling_prob_1:
                    _add_procedure_to_dataset(procedure, given_length, sampled_transitions, result)
            _add_procedure_to_dataset(procedure, proc_length - 1, sampled_transitions, result)
        # else:
        #     # randomly choose the number of steps to retain
        #     retain_num = random.randint(4, len(procedure)-1)
        #     result.append({'given': [procedure[i] for i in range(retain_num)], 'next': ([procedure[i] for i in range(retain_num, len(procedure))]+sampled_transitions)[:6], 'answer': 0})
        #     retain_num_2 = random.randint(4, len(procedure)-1)
        #     while retain_num_2 == retain_num:
        #         retain_num_2 = random.randint(5, len(procedure)-1)
        #     result.append({'given': [procedure[i] for i in range(retain_num_2)], 'next': ([procedure[i] for i in range(retain_num_2, len(procedure))]+sampled_transitions)[:6], 'answer': 0})
    return result


In [ ]:
_train = sampling(train)
_val = sampling(val)
_test = sampling(test)


In [ ]:
len(_train), len(_val), len(_test)

In [ ]:
def get_distribution_of_lengths(arr):
    lengths = [len(x['given']) for x in arr]
    counts = np.bincount(lengths)
    return counts

## get_distribution_of_lengths(_train), get_distribution_of_lengths(_val), get_distribution_of_lengths(_test)

In [ ]:
(len(X_train), len(X_val), len(X_test)), (len(_train), len(_val), len(_test))

In [ ]:
# _train[0]

In [ ]:
 def test_delimiters():
    data_ = get_traversals(X)

    # check if ':::' present in data
    for procedure in data_:
        for transition in procedure:
            for elements in transition:
                if '###' in elements:
                    print(transition)
                    break
    
# test_delimiters()

In [ ]:
def to_text(data):
    result = []
    for procedure in data:
        datapoints = ' ::::: '.join([f'{x.entity}:::{x.property}:::{x.value}' for x in procedure['given']])
        for idx, next in enumerate(procedure['next']):
            result.append(f'{datapoints}\t:::::::\t{next.entity}:::{next.property}:::{next.value}\t:::::::\t{str(int(idx==procedure["answer"]))}')
    return result



In [ ]:
## print(*to_text([_train[0]]), sep='\n')

In [ ]:
# store the data
with open('processed_data/sampled.0_25.new.pkl', 'wb') as f:
    pickle.dump({'train': _train, 'val': _val, 'test': _test}, f)

In [ ]:
def pretty_print_array(arr):
    for idx, item in enumerate(arr):
        print(idx, item, sep='\t')


In [ ]:


pretty_print_array(get_distribution_of_lengths(_train))